# Optimize Hamiltonian basis with orbital optimization

In this tutorial, we will show how to use the `sqd` package to post-process quantum samples using the [self-consistent configuration recovery technique](https://arxiv.org/abs/2405.05068) and then further optimize the ground state approximation using orbital optimization

Refer to [Sec. II A 4](https://arxiv.org/pdf/2405.05068) for a more detailed discussion on this technique.

### Specify the molecule and generate samples

In this example, we will approximate the ground state energy of an $N_2$ molecule and then improve the answer using orbital optimization. This guide studies $N_2$ at equilibrium, which is mean-field dominated. This means the MO basis is already a good choice for our integrals; therefore, we will rotate our integrals **out** of the MO basis in order to illustrate the effects of orbital optimization.

In [1]:
%%capture
import numpy as np
import pyscf
import pyscf.cc
import pyscf.mcscf
from qiskit_addon_sqd.fermion import rotate_integrals

# Specify molecule properties
spin_sq = 0

# Build N2 molecule
mol = pyscf.gto.Mole()
mol.build(
    atom=[["N", (0, 0, 0)], ["N", (1.0, 0, 0)]],
    # basis="6-31g",
    basis="sto-6g",
    symmetry="Dooh",
)

# Define active space
n_frozen = 2
active_space = range(n_frozen, mol.nao_nr())

# Get molecular integrals
scf = pyscf.scf.RHF(mol).run()
num_orbitals = len(active_space)
n_electrons = int(sum(scf.mo_occ[active_space]))
num_elec_a = (n_electrons + mol.spin) // 2
num_elec_b = (n_electrons - mol.spin) // 2
cas = pyscf.mcscf.CASCI(scf, num_orbitals, (num_elec_a, num_elec_b))
mo = cas.sort_mo(active_space, base=0)
hcore, nuclear_repulsion_energy = cas.get_h1cas(mo)
eri = pyscf.ao2mo.restore(1, cas.get_h2cas(mo), num_orbitals)

# Compute exact energy
exact_energy = cas.run().e_tot

The MO basis is already a good basis for this problem, so we will rotate out of that basis in this guide in order to highlight the effect of orbital optimization.

In [2]:
# Rotate our integrals out of MO basis
rng = np.random.default_rng(24)
num_params = (num_orbitals**2 - num_orbitals) // 2  # antisymmetric, specified by upper triangle
k_rot = (rng.random(num_params) - 0.5) * 0.5
hcore_rot, eri_rot = rotate_integrals(hcore, eri, k_rot)

Generate samples

In [3]:
from qiskit_addon_sqd.counts import generate_bit_array_uniform

# Generate random samples
bit_array = generate_bit_array_uniform(10_000, num_orbitals * 2, rand_seed=rng)

### Iteratively refine the samples using SQD and approximate the ground state

In [4]:
from functools import partial

from qiskit_addon_sqd.fermion import SCIResult, diagonalize_fermionic_hamiltonian, solve_sci_batch

# SQD options
energy_tol = 1e-3
occupancies_tol = 1e-3
max_iterations = 5

# Eigenstate solver options
num_batches = 3
samples_per_batch = 20
symmetrize_spin = True
carryover_threshold = 1e-1
max_cycle = 200

# Pass options to the built-in eigensolver. If you just want to use the defaults,
# you can omit this step, in which case you would not specify the sci_solver argument
# in the call to diagonalize_fermionic_hamiltonian below.
sci_solver = partial(solve_sci_batch, spin_sq=0.0, max_cycle=max_cycle)

# List to capture intermediate results
result_history = []


def callback(results: list[SCIResult]):
    result_history.append(results)
    iteration = len(result_history)
    print(f"Iteration {iteration}")
    for i, result in enumerate(results):
        print(f"\tSubsample {i}")
        print(f"\t\tEnergy: {result.energy + nuclear_repulsion_energy}")
        print(f"\t\tSubspace dimension: {np.prod(result.sci_state.amplitudes.shape)}")


result = diagonalize_fermionic_hamiltonian(
    hcore,
    eri,
    bit_array,
    samples_per_batch=samples_per_batch,
    norb=num_orbitals,
    nelec=(num_elec_a, num_elec_b),
    num_batches=num_batches,
    energy_tol=energy_tol,
    occupancies_tol=occupancies_tol,
    max_iterations=max_iterations,
    sci_solver=sci_solver,
    symmetrize_spin=symmetrize_spin,
    carryover_threshold=carryover_threshold,
    callback=callback,
    seed=rng,
)

Iteration 1
	Subsample 0
		Energy: -108.51514815848884
		Subspace dimension: 900
	Subsample 1
		Energy: -107.74386023793795
		Subspace dimension: 784
	Subsample 2
		Energy: -107.7411531716499
		Subspace dimension: 841
Iteration 2
	Subsample 0
		Energy: -108.53322549223714
		Subspace dimension: 625
	Subsample 1
		Energy: -108.57716824917271
		Subspace dimension: 676
	Subsample 2
		Energy: -108.508931350816
		Subspace dimension: 576
Iteration 3
	Subsample 0
		Energy: -108.55902215621444
		Subspace dimension: 625
	Subsample 1
		Energy: -108.5776045917047
		Subspace dimension: 529
	Subsample 2
		Energy: -108.56103466207057
		Subspace dimension: 676
Iteration 4
	Subsample 0
		Energy: -108.56619527937809
		Subspace dimension: 529
	Subsample 1
		Energy: -108.56463009757981
		Subspace dimension: 625
	Subsample 2
		Energy: -108.56630478170408
		Subspace dimension: 784
Iteration 5
	Subsample 0
		Energy: -108.56883735605814
		Subspace dimension: 529
	Subsample 1
		Energy: -108.58150746468066
		Su

### Refine the subspace

To refine the subspace, we will take the CI strings of the SCI state returned by SQD. Other strategies may be used, like taking the union of the CI strings of the batches in the last configuration recovery iteration. You can save information from the recovery iterations by passing a callback function to `run_sqd` (see [Improving energy estimation of a chemistry Hamiltonian with SQD](../tutorials/01_chemistry_hamiltonian.ipynb)).

In [5]:
print(f"Subspace dimension: {np.prod(result.sci_state.amplitudes.shape)}")
print(f"Energy of that batch from SQD: {result.energy + nuclear_repulsion_energy}")

Subspace dimension: 625
Energy of that batch from SQD: -108.58150746468066


### Perform orbital optimization to improve the energy approximation

We now describe how to optimize the orbitals to further improve the quality of the sqd calculation.

The orbital rotations that are implemented in this package are those described by:
$$
U(\kappa) = e^{\sum_{pq, \sigma} \kappa_{pq} c^\dagger_{p\sigma} c_{q\sigma}},
$$
where $\kappa_{p, q} \in \mathbb{R}$ and $\kappa_{p, q} = -\kappa_{q, p}$. The orbitals are optimized to 
minimize the variational energy:
$$
E(\kappa) = \langle \psi | U^\dagger(\kappa) H U(\kappa)  |\psi \rangle,
$$
with respect to $\kappa$ using gradient descent with momentum. Recall that 
$|\psi\rangle$ is spanned in a subspace defined by determinants.

Since the change of basis alters the Hamiltonian, we allow $|\psi\rangle$ to 
respond to the change in the Hamiltonian. This is done by performing a number of alternating
self-consistent optimizations of $\kappa$ and $|\psi\rangle$. We recall that the optimal
$|\psi\rangle$ is given by the lowest eigenvector of the Hamiltonian projected into the
subspace.

The ``sqd.fermion.fermion`` module provides the tools to perform this alternating
optimization. In particular, the function ``sqd.fermion.optimize_orbitals()``.

Some of the arguments that define the optimization are:

- ``num_iters``: number of self-consistent iterations.
- ``num_steps_grad``: number of gradient step updates performed when optimizing 
$\kappa$ on each self-consistent iteration.
- ``learning_rate``: step-size in the gradient descent optimization of $\kappa$.

In [6]:
from qiskit_addon_sqd.fermion import optimize_orbitals

k_flat = (rng.random(num_params) - 0.5) * 0.1
num_iters = 20
num_steps_grad = 10_000  # relatively cheap to execute
learning_rate = 0.05

e_improved, k_flat, orbital_occupancies = optimize_orbitals(
    (result.sci_state.ci_strs_a, result.sci_state.ci_strs_b),
    hcore_rot,
    eri_rot,
    k_flat,
    open_shell=False,
    spin_sq=spin_sq,
    num_iters=num_iters,
    num_steps_grad=num_steps_grad,
    learning_rate=learning_rate,
    max_cycle=max_cycle,
)

Here we see that by optimizing rotation parameters for our Hamiltonian, we can improve the result from SQD.

In [7]:
print(f"Exact energy: {exact_energy}")
print(f"SQD energy: {result.energy + nuclear_repulsion_energy}")
print(f"Energy after OO: {e_improved + nuclear_repulsion_energy}")

Exact energy: -108.59598735098602
SQD energy: -108.58150746468066
Energy after OO: -108.49008547897714
